In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data.shape

(42000, 785)

In [5]:
data = np.array(data)
m , n = data.shape

np.random.shuffle(data)

data_dev  = data[0:1000].T

Y_dev = data_dev[0]
X_dev = data_dev[1: n] /255

data_train = data[1000: m].T

Y_train = data_train[0]
X_train = data_train[1:n] /255

In [6]:
print(X_train.shape)
print(Y_train.shape)
print(X_dev.shape)
print(Y_dev.shape)

(784, 41000)
(41000,)
(784, 1000)
(1000,)


Applying a 2 Layer Neural Network on the MNIST Dataset

In [7]:
def init_params():
    W1 = np.random.rand(10, 784) * 0.01
    b1 = np.zeros((10, 1)) 
    W2 = np.random.rand(10, 10) * 0.01
    b2 = np.zeros((10, 1)) 
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  
    return expZ / np.sum(expZ, axis=0, keepdims=True)

    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis = 1, keepdims = True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis = 1, keepdims = True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [8]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

def compute_cost(A2, Y):
    m = Y.shape[0]
    one_hot_Y = one_hot(Y)
    cost = -np.sum(one_hot_Y * np.log(A2 + 1e-8)) / m
    return np.squeeze(cost)

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            predictions = get_predictions(A2)
            acc = get_accuracy(predictions, Y)
            cost = compute_cost(A2, Y)
            print(f"Iteration {i}: Accuracy = {acc:.4f}, Cost = {cost:.4f}")
    
    return W1, b1, W2, b2


In [9]:
gradient_descent(X_train,Y_train,0.1, 250)

Iteration 0: Accuracy = 0.1020, Cost = 2.3027
Iteration 10: Accuracy = 0.0981, Cost = 2.2953
Iteration 20: Accuracy = 0.0981, Cost = 2.2858
Iteration 30: Accuracy = 0.1097, Cost = 2.2688
Iteration 40: Accuracy = 0.1794, Cost = 2.2381
Iteration 50: Accuracy = 0.2025, Cost = 2.1987
Iteration 60: Accuracy = 0.2165, Cost = 2.1572
Iteration 70: Accuracy = 0.2268, Cost = 2.1160
Iteration 80: Accuracy = 0.2405, Cost = 2.0736
Iteration 90: Accuracy = 0.2695, Cost = 2.0253
Iteration 100: Accuracy = 0.3345, Cost = 1.9643
Iteration 110: Accuracy = 0.4140, Cost = 1.8849
Iteration 120: Accuracy = 0.4508, Cost = 1.7880
Iteration 130: Accuracy = 0.4806, Cost = 1.6808
Iteration 140: Accuracy = 0.5125, Cost = 1.5706
Iteration 150: Accuracy = 0.5488, Cost = 1.4648
Iteration 160: Accuracy = 0.5885, Cost = 1.3690
Iteration 170: Accuracy = 0.6334, Cost = 1.2845
Iteration 180: Accuracy = 0.6667, Cost = 1.2099
Iteration 190: Accuracy = 0.6879, Cost = 1.1419
Iteration 200: Accuracy = 0.7056, Cost = 1.0773
Ite

(array([[0.00616049, 0.00733598, 0.00887002, ..., 0.00632827, 0.00250802,
         0.00694308],
        [0.00109876, 0.00717132, 0.00286802, ..., 0.00520035, 0.0084522 ,
         0.0021769 ],
        [0.00209089, 0.00057212, 0.00918138, ..., 0.00115385, 0.0030072 ,
         0.00691877],
        ...,
        [0.00319624, 0.00606154, 0.00455201, ..., 0.00891039, 0.00279818,
         0.00218595],
        [0.00956473, 0.00207408, 0.00474076, ..., 0.00453081, 0.00281488,
         0.00126895],
        [0.00692158, 0.00478222, 0.00506773, ..., 0.00648505, 0.00540149,
         0.00076267]]),
 array([[-0.01863249],
        [ 0.08351444],
        [-0.05029383],
        [ 0.02749264],
        [ 0.01705548],
        [ 0.03033941],
        [ 0.11893783],
        [-0.02527034],
        [-0.03235502],
        [ 0.04253539]]),
 array([[-2.44967773e-01,  4.64368878e-01, -8.69753833e-02,
          2.49066612e-01, -2.50504071e-01,  6.23606874e-01,
         -2.28703517e-01,  1.37125906e-01,  1.20661394e-0